In [1]:
#라이브러리
import sqlite3

import requests
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
from requests import Session
from bs4 import BeautifulSoup
from selenium import webdriver

import time
import datetime
import pandas as pd
from tqdm import tqdm
import re

/Users/seomin/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
%cd Desktop

/Users/seomin/Desktop


In [3]:
tickers = {
    '친환경' : ['336260', '009830', '086520', '011930', '018000'],
    '서비스업' : ['035720', '035420', '036570', '041140', '018260'],
    '반도체' : ['005930', '000660', '011790', '014680', '000990'],
    '유통업' : ['023530', '004170', '008770', '028260', '026960'],
    '금융' : ['105560', '071050', '055550', '316140', '086790'],
    '메타버스' : ['047080', '089890', '030350', '230980', '069410'],
}
sdate = '2021-08-19'
edate = '2021-10-01'

In [4]:
%mkdir 친환경
%mkdir 서비스업
%mkdir 반도체
%mkdir 유통업
%mkdir 금융
%mkdir 메타버스

mkdir: 서비스업: File exists
mkdir: 반도체: File exists
mkdir: 유통업: File exists
mkdir: 금융: File exists
mkdir: 메타버스: File exists


In [5]:
#예진언니 코드
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

In [6]:
headers={
   
    'Host': 'www.paxnet.co.kr',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Referer': 'http://www.paxnet.co.kr/stock/report/reportView?menuCode=2222&currentPageNo=1&reportId=132756&searchKey=stock&searchValue=',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cookie': 'FBA=163283664230886271; WMONID=5MpxAuP9Taq; _ga=GA1.3.2121813615.1632836643; _gid=GA1.3.298923493.1632992503; __gpi=00000000-0000-0000-0000-000000000000; __igaw__adid=MDAwPTExYWEwODRlLTIxY2QtMTFlYy04NTc2LTAyNDJhYzExMDAwMg==; _pubcid=9cc20c7b-4429-40ff-a605-989ab725a29f; JSESSIONID=03121F2E6A923A7E1C83338B64134FAC; __gads=ID=e98871ef624ba1af-22375635d8ca00a9:T=1632836643:RT=1632992523:S=ALNI_MbMTfoxnsX9QIcZqShvNlWMQjWqrA; wcs_bt=aac9f038ec2a5:1632992895'
}

In [7]:
paxnet = pd.DataFrame(columns = ['Type','TypeName', 'Date', 'Title', 'AppropriatePrice','Investment opinion', 'Offer', 'File'])
paxnet

,Type,TypeName,Date,Title,AppropriatePrice,Investment opinion,Offer,File


In [8]:
#소중하다~

for key in tickers.keys():
    for tk in tqdm(tickers[key]):
        p = 1
        rid = 0
        while 1:            
            url = f"http://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={p}&reportId={rid}&searchKey=stock&searchValue={tk}"
            html = requests.get(url, headers = headers).content
            from bs4 import BeautifulSoup
            soup = BeautifulSoup(html,'lxml')
            if soup.find('div','message-wrap') :
                break
            
            report = soup.find_all('ul')[1]
            type_name = [i.text for i in report.select('strong > a')]
            title = [i.text for i in report.select('p > a')]
         
            #가격
            price = [i.text.replace('적정가격', '').strip() for i in soup.select('ul > li > div:nth-of-type(3)')][1:]
            #투자의견
            opinion = [i.text.replace('투자의견', '').strip() for i in soup.select('ul > li > div:nth-of-type(4)')][1:]
            #제공출처
            offer = [i.text.replace('제공출처', '').strip() for i in soup.select('ul > li > div:nth-of-type(5)')][1:]
            #작성일
            date = [i.text.replace('작성일', '').strip() for i in soup.select('ul > li > div:nth-of-type(6)')][1:]
        
            for i in range(len(title)):
                #pdf 
                rid = re.findall('[0-9]+', report.select('p > a')[i]['href'])[0]
                url =  f"http://www.paxnet.co.kr/stock/report/reportView?menuCode=2222&currentPageNo={p}&reportId={rid}&searchKey=stock&searchValue={tk}"
                report_html = requests.get(url, headers=headers).content
                rsoup = BeautifulSoup(report_html, 'lxml')
                link = rsoup.select('#contents > div.cont-area > div.report-view > div.report-view-file > span > a')[0]['href']
                file = rsoup.select('#contents > div.cont-area > div.report-view > div.report-view-file > span > a')[0].text.replace("\n","")
                
                paxnet.loc[len(paxnet)] = [key, type_name[i], date[i], title[i], price[i], opinion[i], offer[i], file]
                
                # pdf download
                resp = download(f"{link}", headers=headers)
                pdf = open(f'./{key}/{file}', 'wb')
                pdf.write(resp.content)
                pdf.close()
            p=p+1
            
                




 40%|████      | 2/5 [00:51<01:15, 25.16s/it]

KeyboardInterrupt: 

In [ ]:
len(title), len(price), len(opinion), len(offer), len(date)

In [9]:
paxnet

,Type,TypeName,Date,Title,AppropriatePrice,Investment opinion,Offer,File
0,친환경,두산퓨얼셀,2021.07.29,수소경제 방향성은 굳건하다,-원,-,SK증권,595057.pdf
1,친환경,두산퓨얼셀,2021.07.28,2Q21 Review: 필요한 건 시간,"71,000원",매수,메리츠증권,594938.pdf
2,친환경,두산퓨얼셀,2021.07.28,정책 발표 이후 수주 공백 해소 기대,"70,000원",매수,하나금융투자,DFC_210728_2Q21Re.pdf
3,친환경,두산퓨얼셀,2021.04.28,정책이 만든 계절성. 상저하고 기대,"77,000원",매수,하나금융투자,DFC_210428_1Q21Re.pdf
4,친환경,두산퓨얼셀,2021.04.28,1Q21 Review: 거쳐야만 하는 길,"71,000원",매수,메리츠증권,588035.pdf
...,...,...,...,...,...,...,...,...
423,서비스업,엔씨소프트,2018.08.16,신작 모멘텀은 4Q18부터,"430,000원",매수,DB증권,20180816_036570_101033_1064.pdf
424,서비스업,엔씨소프트,2018.08.16,"Lineage, 20년 클래스의 위엄","480,000원",매수,메리츠종금증권,20180816073052317K_02.pdf
425,서비스업,엔씨소프트,2018.07.17,아직은 멀리 있는 신작,"430,000원",Hold,DB증권,20180717_036570_101033_1054.pdf
426,서비스업,엔씨소프트,2018.07.16,신작 부재 속 분할 매수 권고,"505,000원",매수(유지),HI투자증권,[15194422]_036570.pdf
